In [1]:
import networkx as nx
import pylab as plt
import pandas as pd
import numpy as np
# Create blank graph
D=nx.DiGraph()

citations_with_data= pd.read_csv('citations(hep-th)_with_Data.csv')

C:\Users\allen\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [17]:
DF = pd.read_csv('./AM_Matrix.csv')
DF.rename(columns={'Unnamed: 0':'From'}, inplace = True)
DF.set_index('From',inplace = True,drop=True)

DF

,00168ef7c9a4fed70901c32e2a3311a60893e328859fcd615100cf40da97344e,00a5614eecf54e2d713de00b7a6a75814ca14d5732779117f77346fa01ddca12,00ef89995ea4bb4aa275935f61e7a0e088efbc28666464b11d6dd409616db1a2,00f1809b78bdb4854c678a8900fca8c0454b78bc5a41811c40f97a0ab61ae5a0,010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f653f755ec24802635e3a,012339d7d671c04790fe138a9a3b12cf8b086871a5d3f30a3e7bc8397dc90f12,018ebf6145dd4fd0a4d65369c1eb0a2c349c0c46036f46d5791fce5d7c49d08a,01ae695ad5641c0f2b920d0066852d6fe8064acd3fe7590f6230bafcd6ff534a,023e3d30c4715150b21baa4a976d36f4647f21d15f17b9703231c7d51dafedb0,02b06f1e5caa2d6b8489ffa40bb321c7b62cdf2bce0d6ef61cc38bffa2a4f8cc,...,fdd022bf453a8d620dbf0c05b7a01c1233ec15454484aa3c2dce65e13489ef9d,fde67c167fea57ba49205b24abc47bafc96f76d318e19e9183314120307dc4b6,fde7b9779edadb0549355d89d9ec330250f09f965b059ef1317659c1255ae3d8,fdfa6a73ea5fbfcbd5c7650951ec26f5fbea28d8d4b275b929673a360947b7d7,fe134f3e076ab28cdb553a309ef1940a3e3c5d454ec26b28793d1e9eb042dad5,fe8853528bdee7911b91829edffd3479ee6376168c8790665ec59f1ecde8b05c,feca281ea68a9c47ec6fb59ec5d9f31d76105ad9d058a0d20dd37f5b97d848b1,ff24acad38556b1c8d521f449c0399008632be8d96b0c25a3da0766658502646,ff665fdeae13e20a855549e1e389b3992088650b2feea22b4dbfbe0c051d5ccc,ff9af28010b1a78e3af698c64734cecf907988b927435d9a887a4cdcce248611
From,,,,,,,,,,,,,,,,,,,,,
00168ef7c9a4fed70901c32e2a3311a60893e328859fcd615100cf40da97344e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00a5614eecf54e2d713de00b7a6a75814ca14d5732779117f77346fa01ddca12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00ef89995ea4bb4aa275935f61e7a0e088efbc28666464b11d6dd409616db1a2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00f1809b78bdb4854c678a8900fca8c0454b78bc5a41811c40f97a0ab61ae5a0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f653f755ec24802635e3a,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe8853528bdee7911b91829edffd3479ee6376168c8790665ec59f1ecde8b05c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feca281ea68a9c47ec6fb59ec5d9f31d76105ad9d058a0d20dd37f5b97d848b1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ff24acad38556b1c8d521f449c0399008632be8d96b0c25a3da0766658502646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
authors = DF.columns.values
len(authors)

878

In [18]:
DF.sum()

00168ef7c9a4fed70901c32e2a3311a60893e328859fcd615100cf40da97344e    0
00a5614eecf54e2d713de00b7a6a75814ca14d5732779117f77346fa01ddca12    4
00ef89995ea4bb4aa275935f61e7a0e088efbc28666464b11d6dd409616db1a2    4
00f1809b78bdb4854c678a8900fca8c0454b78bc5a41811c40f97a0ab61ae5a0    1
010d2592c5ebdfdb9828fdf01e0f0e0a2c35734e9f0f653f755ec24802635e3a    2
                                                                   ..
fe8853528bdee7911b91829edffd3479ee6376168c8790665ec59f1ecde8b05c    1
feca281ea68a9c47ec6fb59ec5d9f31d76105ad9d058a0d20dd37f5b97d848b1    0
ff24acad38556b1c8d521f449c0399008632be8d96b0c25a3da0766658502646    0
ff665fdeae13e20a855549e1e389b3992088650b2feea22b4dbfbe0c051d5ccc    1
ff9af28010b1a78e3af698c64734cecf907988b927435d9a887a4cdcce248611    5
Length: 878, dtype: int64

In [21]:
(DF != 0).sum(1).sum() # mi dice quanti sono

1869

In [23]:
W = 0
for i in authors:
    W += DF.loc[i].sum()
    
W #total weight

1938

In [19]:

def scaleValues(E_matrix):
    #calcolo il total weight sul grafo
    W = 0
    for i in E_matrix.columns.values:
        W += E_matrix.loc[i].sum()
    print(f'Total weight: {W}')
    
    def scale(x):
        return x/W
    
    E_matrix = E_matrix.applymap(scale)
    
    return E_matrix


    

    



def calcolaMerge(E_matrix):
    
    E_matrix = scaleValues(E_matrix)
    
    #print(f'E_matrix scalata:\n {E_matrix}')
    

    #singola iterazione per calcolare join tra communities
    Q = 0

    #calcolo Q iniziale 
    for i in E_matrix.columns.values:
        a_i_square = pow(E_matrix.loc[i].values.sum(),2)
        e_ii = E_matrix.loc[i][i]
        Q += e_ii - a_i_square

    print(f'Q iniziale: {Q}')

    #definisco df in cui memorizzare DQs
    df = pd.DataFrame(columns=['pair','DQ','newQ'])
    pairs_to_try = []

    #individuo le coppie da testare => coppie con almeno un arco tra loro
    for i in E_matrix.columns.values:

        temp = pd.DataFrame(E_matrix.loc[i])    
        possible_dest = temp[temp[i]>0].index.values

        for j in possible_dest:
            pairs_to_try.append((i,j))

    #calcolo DQ per ogni coppia individuata
    for k in pairs_to_try:
        i = k[0]
        j = k[1]

        a_i = E_matrix.loc[i].values.sum()
        a_j = E_matrix.loc[j].values.sum()

        DQ = 2 * (E_matrix.loc[i][j] - (a_i*a_j))
        #delta_Qs.append(DQ)

        last = len(df)
        df.loc[last,'pair'] = k #aggiungo valori
        df.loc[last,'DQ'] = DQ #aggiungo valori
        df.loc[last,'newQ'] = Q +DQ #aggiungo valori


    print(f'DQ ottenute dai merge:\n {df}')

    #identifico la coppia che fornisce maggior incremento positivo in Q => Q + DQ maggiore
    winning_pair = df.query('newQ == newQ.max()')['pair'].values[0]

    print(f'Miglior merge: {winning_pair}')


    #genero le nuove colonne merged
    old_columns = E_matrix.columns.values

    print(f'old_columns: {old_columns}')


    old_columns = np.delete(old_columns, np.argwhere(old_columns == winning_pair[0]))
    old_columns = np.delete(old_columns, np.argwhere(old_columns == winning_pair[1]))


    new_columns = np.append(old_columns,winning_pair[0]+winning_pair[1])
    print(f'new_columns: {new_columns}')

    #genero le nuove rows merged



    #rows to merge
    merged_row = E_matrix.loc[winning_pair[0]] + E_matrix.loc[winning_pair[1]]

    #add new merged row to matrix
    E_matrix.loc[winning_pair[0]+winning_pair[1]] = merged_row

    #remove rows of single merged communities
    E_matrix.drop(index = winning_pair[0], inplace=True)
    E_matrix.drop(index = winning_pair[1], inplace=True)


    #NOW CHANGE IN THE COLUMNS

    #first compute the list of values to be insterted in the new column
    new_values = []

    #for all rows compute the merged sum value 
    for i in new_columns:
        new_value = E_matrix.loc[i][[winning_pair[0],winning_pair[1]]].sum()
        new_values.append(new_value)

    #remove the single columns of merged comm
    E_matrix = E_matrix.drop([winning_pair[0], winning_pair[1]], axis=1)

    #add the new merged column
    #E_matrix = E_matrix.insert(winning_pair[0]+winning_pair[1], new_values)


    E_matrix[winning_pair[0]+winning_pair[1]] = new_values

    print(f'AM finale: \n {E_matrix}')

In [14]:
#definisco df con tante colonne quanti autori
E_matrix = pd.DataFrame(columns=['a','b','c'])


E_matrix.loc['a',['a','b','c']] = [0,1,2]
E_matrix.loc['b',['a','b','c']] = [2,0,3]
E_matrix.loc['c',['a','b','c']] = [1,1,0]

E_matrix

,a,b,c
a,0,1,2
b,2,0,3
c,1,1,0


In [15]:
E_matrix2 = calcolaMerge(E_matrix)
E_matrix2

Total weight: 10
E_matrix scalata:
      a    b    c
a  0.0  0.1  0.2
b  0.2  0.0  0.3
c  0.1  0.1  0.0
Q iniziale: -0.38
DQ ottenute dai merge:
      pair    DQ  newQ
0  (a, b)  -0.1 -0.48
1  (a, c)  0.28  -0.1
2  (b, a)   0.1 -0.28
3  (b, c)   0.4  0.02
4  (c, a)  0.08  -0.3
5  (c, b)   0.0 -0.38
Miglior merge: ('b', 'c')
old_columns: ['a' 'b' 'c']
new_columns: ['a' 'bc']
AM finale: 
       a   bc
a   0.0  0.3
bc  0.3  0.4


In [20]:

DF2 = calcolaMerge(DF)
DF2

Total weight: 1938
Q iniziale: 0.026512283257771278
DQ ottenute dai merge:
                                                    pair        DQ      newQ
0     (00168ef7c9a4fed70901c32e2a3311a60893e328859fc...  0.001032  0.027544
1     (00a5614eecf54e2d713de00b7a6a75814ca14d5732779...  0.001028  0.027541
2     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001032  0.027544
3     (00ef89995ea4bb4aa275935f61e7a0e088efbc2866646...  0.001026  0.027538
4     (00f1809b78bdb4854c678a8900fca8c0454b78bc5a418...  0.001029  0.027542
...                                                 ...       ...       ...
1864  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001032  0.027544
1865  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001021  0.027534
1866  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001019  0.027531
1867  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001021  0.027534
1868  (ff9af28010b1a78e3af698c64734cecf907988b927435...  0.001027  0.027539

[1869 rows 

KeyError: "['53a49de7833c00df72e5cd19d3e60b94658a426a9fe1a83b01709a458769c8dd'] not found in axis"

# DEVO RENDERE LA MATRICE DI INPUT DF COME E_MATRIX => IL CAMPO FROM NON CI DEVE ESSERE